**steps**
1. identify row with headers
2. map headers to apppriate column number
3. read the line above (a) component till total/sub-total
4. remove all rows with null (expect for yeild)
5. add to full_data


In [1]:
import numpy as np 
import pandas as pd

In [308]:
def read_excel_file(file_path):
    file_ext = file_path.split(".")[-1].lower()
    if file_ext == "xlsb":
        try:
            return pd.read_excel(file_path, sheet_name=None, engine="pyxlsb", dtype=str)
        except Exception as e:
            print(f"❌ Error reading {file_path} (XLSB format): {e}")
            return None
    elif file_ext in ["xls", "xlsx"]:
        try:
            return pd.read_excel(file_path, sheet_name=None, dtype=str)
        except Exception as e:
            print(f"❌ Error reading {file_path} (XLS/XLSX format): {e}")
            return None
    else:
        print(f"⚠️ Unsupported file format: {file_path}")
        return None


In [ ]:
path = r"C:\\Users\\vaibh\\Documents\\IPYNB_notebooks\\interview\\data\\data\\Trust Mutual Fund\\TRUSTMF-Monthly-Portfolio-Report-as-on-28.02.2025.xls"


In [385]:
all_sheets = []

df_raw = read_excel_file(path)
if df_raw is None:
    pass
for sheet_name, sheet_df in df_raw.items():
    # if sheet_name not in sheets_to_avoid:
    all_sheets.append(sheet_df)

        

In [471]:
df_raw.items()

dict_items([('TRUSTMF Banking & PSU Fund',        MONTHLY PORTFOLIO STATEMENT AS ON Feb 28, 2025 Unnamed: 1 Unnamed: 2  \
0   [Pursuant to Regulation 59 A of Securities and...        NaN        NaN   
1                                                 NaN        NaN        NaN   
2   TRUSTMF Banking & PSU Fund\n(An open ended deb...        NaN        NaN   
3                                  Name of Instrument       ISIN     Rating   
4                                    Debt Instruments        NaN        NaN   
..                                                ...        ...        ...   
94  14) Whether during the period any investment i...        NaN        NaN   
95  15) Instances of Fair valuation of securities ...        NaN        NaN   
96  16) Any deviation from the mandated assets all...        NaN        NaN   
97  17) Deviation from the valuation prices given ...        NaN        NaN   
98  18) For other details please refer to our webs...        NaN        NaN   

   Unnam

In [386]:
df = all_sheets[0]
df = df.dropna(how='all')


In [387]:
def fetch_header_row(df :pd.DataFrame) -> list[str]: 
    rows = df.astype(str).agg(' '.join, axis=1)
    idx = rows[rows.apply(lambda x: "instrument" in x.lower())].index.tolist()[0]
    header_row = df.iloc[idx,:].fillna("NULL")
    header_row = [(header_row.iloc[col]) for col in range(header_row.shape[0])]
    return header_row

In [388]:
header_row = fetch_header_row(df)
header_row

['Name of Instrument',
 'ISIN',
 'Rating',
 'Quantity',
 'Market Value (In Rs. lakh)',
 '% To Net Assets',
 'Yield to Maturity (YTM)',
 'Yield to Call \n(YTC) ^^',
 'NULL',
 'NULL',
 'NULL',
 'NULL']

## **Step 2** 

In [389]:
base_headers = ["Name of Instrument","ISIN" , "Industry" , "Yield" , "Quantity" , "Market Value" , "Net Asset Value (NAV)"]

In [390]:
import requests
def generate_embedding(text:str) -> list[float]:
    url = "https://lamhieu-lightweight-embeddings.hf.space/v1/embeddings"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    data = {
        "model": "snowflake-arctic-embed-l-v2.0",
        "input": text
    }

    response = requests.post(url, headers=headers, json=data)
    if response.ok:
        return response.json()["data"][0]["embedding"]
    else:
        raise Exception("No response")

In [391]:
test_header_row=["Company/Issuer/Instrument Name", "Industry/Rating","Quantity","ISIN","Gibbrish" , "Exposure/Market Value", "yeild to Instrument","% to Nav", "Cupon"]

In [392]:
def header_mapper(header_row , * , base_headers = ["Name of Instrument","ISIN" , "Industry" , "Yield" , "Quantity" , "Market Value" , "Net Asset Value (NAV)"]) -> {str:int}:
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity

    header_map = dict()

    base_embeddings = np.array([generate_embedding(value) for value in base_headers])
    header_row_embeddings = np.array([generate_embedding(value) for value in header_row])
    # Compute cosine similarity (shape: 5 x 10)
    similarity_matrix = cosine_similarity(base_embeddings, header_row_embeddings)

    # For each base vector, find the index of the most similar header
    most_similar_indices = np.argmax(similarity_matrix, axis=1)

    # Optionally, get the similarity score too
    most_similar_scores = np.max(similarity_matrix, axis=1)

    # Print results
    for i, (idx, score) in enumerate(zip(most_similar_indices, most_similar_scores)):
        print(f"Base vector {i} ie {base_headers[i]} is most similar to header {idx} ie {header_row[idx]} with score {score:.4f}")
        header_map[base_headers[i]] = int(idx)
        
    return header_map

In [321]:
header_mapper(test_header_row)

Base vector 0 ie Name of Instrument is most similar to header 0 ie Company/Issuer/Instrument Name with score 0.7506
Base vector 1 ie ISIN is most similar to header 3 ie ISIN with score 1.0000
Base vector 2 ie Industry is most similar to header 1 ie Industry/Rating with score 0.6796
Base vector 3 ie Yield is most similar to header 6 ie yeild to Instrument with score 0.5246
Base vector 4 ie Quantity is most similar to header 2 ie Quantity with score 1.0000
Base vector 5 ie Market Value is most similar to header 5 ie Exposure/Market Value with score 0.6699
Base vector 6 ie Net Asset Value (NAV) is most similar to header 7 ie % to Nav with score 0.4694


{'Name of Instrument': 0,
 'ISIN': 3,
 'Industry': 1,
 'Yield': 6,
 'Quantity': 2,
 'Market Value': 5,
 'Net Asset Value (NAV)': 7}

In [393]:
header_map = header_mapper(header_row)
header_map

Base vector 0 ie Name of Instrument is most similar to header 0 ie Name of Instrument with score 1.0000
Base vector 1 ie ISIN is most similar to header 1 ie ISIN with score 1.0000
Base vector 2 ie Industry is most similar to header 3 ie Quantity with score 0.4296
Base vector 3 ie Yield is most similar to header 6 ie Yield to Maturity (YTM) with score 0.6161
Base vector 4 ie Quantity is most similar to header 3 ie Quantity with score 1.0000
Base vector 5 ie Market Value is most similar to header 4 ie Market Value (In Rs. lakh) with score 0.6874
Base vector 6 ie Net Asset Value (NAV) is most similar to header 5 ie % To Net Assets with score 0.5672


{'Name of Instrument': 0,
 'ISIN': 1,
 'Industry': 3,
 'Yield': 6,
 'Quantity': 3,
 'Market Value': 4,
 'Net Asset Value (NAV)': 5}

## **step 3**

In [323]:
rows = df.fillna("").astype(str).agg(' '.join, axis=1)
referance_rows = np.array(rows[rows.apply(lambda x : "stock exchange" in x.lower())].index.tolist())
investment_type_idxes = referance_rows -1
start_row_idxes = referance_rows +1

In [374]:
col_name = df.columns[header_map["Name of Instrument"]]

# All columns except 'Name'
other_cols = df.drop(columns=[col_name],axis =0).columns

# Mask where other columns are all NaN and 'Name' is not NaN
mask = df[other_cols].isna().all(axis=1) & df[col_name].notna()

result = df[mask]
print(result.fillna("").astype(str).agg("".join,axis = 1))


0     [Pursuant to Regulation 59 A of Securities and...
2     TRUSTMF Banking & PSU Fund\n(An open ended deb...
4                                      Debt Instruments
5       (a) Listed / awaiting listing on Stock Exchange
6                                 Govt Securities / SDL
9                            Non Convertible Debentures
20                      (b) Privately placed / Unlisted
22                                 (c) Securitised Debt
26                             Money Market Instruments
27                              Certificate of Deposits
31                                    Commercial Papers
34                                       Treasury Bills
37                               Tri Party Repo (TREPs)
42                                               Others
49                            **  Non Traded Securities
50                                # Unlisted Securities
51    ^^  As per AMFI Best Practices Circular dated ...
52    $$  Investment by Mutual Fund Schemes in u

In [395]:
start_indexes = df.index[mask].to_numpy()
start_indexes

array([ 0,  2,  4,  5,  6,  9, 20, 22, 26, 27, 31, 34, 37, 42, 49, 50, 51,
       52, 55, 56, 57, 72, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95,
       96, 97, 98])

In [324]:
prefinal_end_rows = np.array(rows[rows.apply(lambda x : "total" in x.lower() and "subtotal" not in x.lower())].index.tolist())

In [325]:
referance_end_rows = np.array([prefinal_end_rows[prefinal_end_rows > start_idx][0] for start_idx in start_row_idxes])
valid_ranges = np.array(tuple(zip(start_row_idxes,referance_end_rows)))
valid_ranges

array([[ 6, 24]])

In [328]:
df[valid_ranges[0][0]:valid_ranges[0][1]]

,"MONTHLY PORTFOLIO STATEMENT AS ON Feb 28, 2025",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
7,7.04% GOI - 03-JUN-2029,IN0020240050,SOVEREIGN,2200000,2232.5952,0.10184518992276194,0.066308,NaN,NaN,NaN,NaN,NaN
8,Subtotal,NaN,NaN,NaN,2232.5952,0.10184518992276194,NaN,NaN,NaN,NaN,NaN,NaN
9,Non Convertible Debentures,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,7.46% Indian Railway Finance Corporation Limit...,INE053F08387,CRISIL AAA,2000000,2004.482,0.09143925866487472,0.07392,NaN,NaN,NaN,NaN,NaN
11,7.59% National Housing Bank 14-JUL-2027 **,INE557F08FY4,CRISIL AAA,1900000,1905.3922,0.0869190395492874,0.074425,NaN,NaN,NaN,NaN,NaN
12,7.70% Housing Development Finance Corporation ...,INE040A08955,CRISIL AAA,1900000,1903.4276,0.08682941960379872,0.076175,NaN,NaN,NaN,NaN,NaN
13,7.64% National Bank for Agriculture and Rural ...,INE261F08EJ7,ICRA AAA,1800000,1809.972,0.08256621804744599,0.07485,NaN,NaN,NaN,NaN,NaN
14,7.68% Small Industries Development Bank of Ind...,INE556F08KQ2,CRISIL AAA,1800000,1804.4046,0.08231224773058067,0.0756,NaN,NaN,NaN,NaN,NaN
15,7.75% LIC Housing Finance Limited 23-AUG-2029 **,INE115A07QU9,CRISIL AAA,1800000,1804.005,0.082294019017246,0.0767,NaN,NaN,NaN,NaN,NaN
16,7.60% Power Finance Corporation Limited 13-APR...,INE134E08MX3,CRISIL AAA,1700000,1707.1825,0.07787722823435056,0.074825,NaN,NaN,NaN,NaN,NaN


## **step 4** read all lines in valid ranges

In [301]:
full_data = np.array([0]*(len(base_headers) + 3)) # baseheaders + type + scheme_name + amc_name

In [397]:
investment_types = [rows[investment_type_idx].strip() for investment_type_idx in start_indexes]
investment_types[:5]

['[Pursuant to Regulation 59 A of Securities and Exchange Board of India (Mutual Funds)',
 'TRUSTMF Banking & PSU Fund\n(An open ended debt scheme predominantly investing in debt instruments of banks, Public Sector Undertakings,Public Financial Institutions and Municipal Bonds)',
 'Debt Instruments',
 '(a) Listed / awaiting listing on Stock Exchange',
 'Govt Securities / SDL']

In [465]:
full_data = pd.DataFrame(columns= list(header_map.keys()) + ["type" , "scheme" , "amc name"])

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
for i in range(len(start_indexes)):
    start_index = start_indexes[i]
    if i != len(start_indexes)-1:
        end_index = start_indexes[i+1]
    else :
        end_index = len(df)
    
    for (index,row) in df.iloc[start_index:end_index].iterrows():
        
        values = header_map.copy()
        for (key , idx) in header_map.items():
            values[key] = row[idx]
        isin = values['ISIN']
        if not str(isin).lower().startswith("in"):
            continue
        print(f"{index} ",end=" , ") # just to keep track

        full_data = pd.concat([full_data , pd.DataFrame([values])],ignore_index=True)
print("sheet over")
 
        
        
        

7  , 10  , 11  , 12  , 13  , 14  , 15  , 16  , 17  , 18  , 28  , 29  , 32  , 35  , 43  , sheet over


In [501]:
start_indexes

array([ 0,  2,  4,  5,  6,  9, 20, 22, 26, 27, 31, 34, 37, 42, 49, 50, 51,
       52, 55, 56, 57, 72, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95,
       96, 97, 98])

In [507]:
full_data = full_data.drop_duplicates()

In [508]:
len(full_data)

15